# 权限配置

In [1]:
import sagemaker
import os
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::847380964353:role/spot-bot-SpotSageMakerExecutionRole-TP8BLT3Z5JJL
sagemaker bucket: sagemaker-us-west-2-847380964353
sagemaker session region: us-west-2


# 数据准备

In [2]:
# dataset used
dataset_name = 'data'
# s3 key prefix for the data
s3_prefix = 'datasets/data'
WORK_DIRECTORY = './data/'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=s3_prefix)
data_location

's3://sagemaker-us-west-2-847380964353/datasets/ruanhua'

# 超参数定义

In [5]:
from sagemaker.huggingface import HuggingFace

# hyperparameters which are passed to the training job
hyperparameters={'reference_column':'ref',
                 'hypothesis_column':'hyp',
                 'train_file':'/opt/ml/input/data/train/parasci_train.csv',
                 'validation_file':'/opt/ml/input/data/test/parasci_val.csv',
                 'output_dir':'/opt/ml/model',
                 'do_train':True,
                 'do_eval':True,
                 'max_source_length': 128,
                 'max_target_length': 128,
                 'model_name_or_path': 't5-large',
                 'learning_rate': 3e-4,
                 'num_train_epochs': 1,
                 'per_device_train_batch_size': 2,#16
                 'gradient_accumulation_steps':2, 
                 'save_strategy':'epoch',
                 'evaluation_strategy':'epoch',
                 'save_total_limit':1,
                 }
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}
# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='run_paraphrase.py',
        source_dir='./scripts',
        instance_type='ml.p3.2xlarge',#'ml.p3dn.24xlarge'
        instance_count=1,
        role=role,
        max_run=24*60*60,
        transformers_version='4.6',
        pytorch_version='1.7',
        py_version='py36',
        volume_size=128,
        hyperparameters = hyperparameters,
#         distribution=distribution
)

# 模型训练

In [6]:
huggingface_estimator.fit(
  {'train': data_location+'/parasci_train.csv',
   'test': data_location+'/parasci_val.csv'}
)

2022-04-19 05:56:08 Starting - Starting the training job...ProfilerReport-1650347767: InProgress
...
2022-04-19 05:56:51 Starting - Preparing the instances for training......
2022-04-19 05:57:54 Downloading - Downloading input data...
2022-04-19 05:58:32 Training - Downloading the training image...................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-04-19 06:01:44,067 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-04-19 06:01:44,093 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-04-19 06:01:44,102 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-04-19 06:01:44,485 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Attempting uninstall: regex
    Found existing installation: regex 2021.4.4

# 模型加载&部署

In [13]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
#    env= {'HF_TASK':'text-generation'},
   model_data="s3://sagemaker-us-west-2-847380964353/huggingface-pytorch-training-2022-04-19-05-56-07-474/output/model.tar.gz",  # path to your trained SageMaker model
   role=role,                                            # IAM role with permissions to create an endpoint
   transformers_version="4.6",                           # Transformers version used
   pytorch_version="1.7",                                # PyTorch version used
   py_version='py36',                                    # Python version used
    
)

In [14]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)

------------!

# endpoint调用

In [8]:
from sagemaker.huggingface.model import HuggingFacePredictor
predictor=HuggingFacePredictor(endpoint_name='huggingface-pytorch-inference-2022-04-19-06-41-55-309')

In [30]:
import time
s=time.time()
profix=['Low level paraphrase:','Medium level paraphrase:','High level paraphrase:']
for i in profix:
    out=predictor.predict({
        'inputs': [i+"Part of why it’s so difficult to begin is that we are dreading the task."],
        "parameters": {"max_length": 256},
    })
    break
e=time.time()
print(e-s)


0.7857880592346191


In [31]:
len(out[0]['generated_text'].split(' '))

15

# 本地训练

In [ ]:
!python scripts/run_paraphrase.py \
    --model_name_or_path t5-small \
    --do_train \
    --do_eval \
    --train_file data/parasci_train.csv \
    --validation_file data/parasci_val.csv \
    --output_dir /tmp/tst-summarization \
    --overwrite_output_dir \
    --save_strategy 'epoch' \
    --reference_column 'ref' \
    --num_train_epochs 1 \
    --hypothesis_column 'hyp' \
    --max_source_length 128 \
    --output_dir models \
    --max_target_length 128 \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate

In [7]:
!pip install -r scripts/requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 KB 4.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 KB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 KB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 KB 10.9 MB/s eta 0:00:00
